In [49]:
# No function support

# manual push pops
# Calling a function with X methods
# some basic comparisons 
# quit 
# constants support

import json

filename = "lk_get_craw.bdx"
bdx = open(filename).read().split("\n")
name = filename.split(".")[0]

PRINT_ASSEMBLY = True

In [53]:
def doConst(management, line_syntax):
    try:
        name = line_syntax[1]
        value = int(line_syntax[2])
    except:
        print("ParseError error converting {} to int".format(line_syntax))
        value = int(line_syntax[1])
    if name in management["constants"]:
        raise Exception("ConstantError: Constant already defined {}".format(name))
    management["constants"][name] = value

def doPush(management, line_syntax=None, value=None):
    if not value:
        value = int(line_syntax[1])
    management["instructions"].append("pushImm {}".format(value))

def doPop(management, line_syntax=None, value=None):
    raise Exception("Not Implemented")

def doCall(management, line_syntax):
    ls = ''.join(line_syntax[1:]).split("(")
    trap_name = ls[0]
    arguments = [a for a in ls[1].split(")")[0].split(",") if len(a)]
    for arg in arguments:
        argument = arg
        if arg in management["constants"]:
            argument = management["constants"][arg]
        doPush(management, value=argument)
    management["instructions"].append(trap_name)

def doQuit(management, line_syntax):
    management["instructions"].append("ret")

def doIf(management, line_syntax):
    raise Exception("Not Implemented")

def doLabel(management, line_syntax):
    label_name = ' '.join(line_syntax[1:])
    management["end_labels"].append(label_name)
    
def assembleBDX(instructions):
    pass

In [54]:
commands = {
    "constant": doConst,
    "push": doPush,
    "pop": doPop,
    "call": doCall,
    "quit": doQuit,
    "If": doIf,
    "label": doLabel
}

management = {
    "sizes": {
        "workSize": 0,
        "stackSize": 512,
        "termSize": 512
    },
    "constants": {},
    "instructions": [],
    "end_labels": [
    ]
}

for line in bdx:
    # Allow python style comments
    if len(line.strip()) == 0:
        continue
    elif line.startswith("#"):
        continue
    elif '#' in line:
        line = line.split("#")[0]

    if line.startswith("workSize:"):
        management["sizes"]["workSize"] = int(line.split(" ")[1])
        continue
    elif line.startswith("stackSize:"):
        management["sizes"]["stackSize"] = int(line.split(" ")[1])
        continue
    elif line.startswith("termSize:"):
        management["sizes"]["termSize"] = int(line.split(" ")[1])
        continue

    line_syntax = line.strip().split(" ")
    command = line_syntax[0]
    if command not in commands:
        raise Exception("SyntaxError: Command not found: {}".format(command))
    commands[command](management, line_syntax)

if management["instructions"][-1] != "ret":
    management["instructions"].append("ret")


if PRINT_ASSEMBLY:
    print("\n".join(["{}: {}".format(k,v) for k,v in management["sizes"].items()]))
    print("\n")
    print("\n".join(management["instructions"]))
    print("\n")
    print("\n".join(management["end_labels"]))


# Now create the BDX off the assembly
assembleBDX(management["instructions"])

workSize: 0
stackSize: 512
termSize: 512


pushImm 7
pushImm 525
trap_saveram_set_form_ability
ret


jump_start
jump_end


In [67]:
instructions = json.load(open("instructions.json"))
traps = json.load(open("traps.json"))
instructions = {
    "pushImm": [0x80, 0],
    "ret": [0x89, 0]
}
traps = json.load(open("traps.json"))

hexdump = bytearray(name, "utf-8")
while len(hexdump) < 16:
    hexdump.append(0) # delimiter

import struct

hexdump += bytearray(struct.pack("<i", management["sizes"]["workSize"]))
hexdump += bytearray(struct.pack("<i", management["sizes"]["stackSize"]))
hexdump += bytearray(struct.pack("<i", management["sizes"]["termSize"]))

# Just a default triggers section, will need to be real for more advanced stuff
hexdump += bytearray([2, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

for instruction in management["instructions"]:
    if instruction.startswith("trap"):
        hexdump += bytearray([0xa,0])
        trap = traps[instruction]
        hexdump += bytearray(int(trap["tableIdx"]))
        hexdump += bytearray(struct.pack("<h", int(trap["funcIdx"])))
    else:
        split = instruction.split(" ")
        instr = split[0]
        if instr not in instructions:
            raise Exception("Instruction not found: {}".format(instr))
        hexdump += bytearray(instructions[instr])
        if instr == "pushImm":
            hexdump += bytearray(struct.pack("<i", int(split[1])))

# for l in range(len(management["end_labels"])):
#     label = management["end_labels"][l]
#     hexdump += bytearray(name, "utf-8")
#     hexdump += bytearray([0,0]) 

# hacking the end together
hexdump += bytearray([106, 117, 109, 112, 32, 115, 116, 97, 114, 116, 0, 0, 106, 117, 109, 112, 32, 101, 110, 100, 0, 0])

hexdump == correct
# str([i for i in hexdump])
# for i in range(len(correct)):
#     x = correct[i]
#     y = hexdump[i] if i < len(hexdump) else '_'
#     print("{} {}".format(x, y))

True

In [46]:
hex(0xa)

'0xa'

In [44]:
correct = open("lk_g.bdx", "rb").read()
correct
str([i for i in bytearray(correct)])

'[108, 107, 95, 103, 101, 116, 95, 99, 114, 97, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 0, 7, 0, 0, 0, 128, 0, 13, 2, 0, 0, 10, 0, 98, 0, 137, 0, 106, 117, 109, 112, 32, 115, 116, 97, 114, 116, 0, 0, 106, 117, 109, 112, 32, 101, 110, 100, 0, 0]'

In [ ]:
'''
'   '0x6a', '0x75', '0x6d', '0x70', '0x20', '0x73', '0x74', '0x61', '0x72', '0x74', '0x0', '0x0', '0x6a', '0x75', '0x6d', '0x70', '0x20', '0x65', '0x6e', '0x64', '0x0', '0x0'
# 0x80', '0x0',                '0x7', '0x0', '0x0', '0x0',
pushImm 7
#'0x80', '0x0',                '0xd', '0x2', '0x0', '0x0',
pushImm 525
# '0xa', '0x0',                '0x62', '0x0',
trap_saveram_set_form_ability
# '0x89', '0x0',
ret
'''

In [ ]:
# break this for assembly instructions
int v0 = br.ReadUInt16();
                        int opc = v0 & 15;
                        int sub = (v0 >> 4) & 3;
                        var ssub = v0 >> 6;

                        Instr instr;
                        try {
                            instr = pcode.Instr.Single(
                                it => it.opcode == opc
                                && (it.sub == -1 || it.sub == sub)
                                && (it.ssub == -1 || it.ssub == ssub)
                            );
                        }


In [31]:
v0 = 0x89

In [32]:
v0

137

In [6]:
hexdump

b'lk_get_craw'

In [7]:
len(hexdump)

11

In [12]:
hexdump+=bytes(0x0)

In [13]:
len(hexdump)

11

In [14]:
hexdump

b'lk_get_craw'

In [15]:
hexdump += 0x0

TypeError: can't concat int to bytes

In [30]:
7 & 8

0